In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('../../src'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

In [1]:
import csv 
from pathlib import Path
from os import listdir
import pickle

from labeling_utils import load_labels

import numpy as np
from sklearn.metrics import precision_recall_fscore_support


tags=["Songbird","Water Bird","Insect","Running Water","Rain","Cable","Wind","Vehicle","Aircraft"]
from tabulate import tabulate
tag_set=tags[:]


* Count only highest ranking tag (what if both of them exist)
* Bird tag is confusing, it can be also waterbird, how to handle ? Also Animal
* 

In [2]:
#LOAD MODEL predictions
splits_path= Path('/files/scratch/enis/data/nna/labeling/splits/')

labelsbymodelpath=Path('/scratch/enis/data/nna/labeling/deep_labels/')
dbfile = open(labelsbymodelpath / "classifications_all_2.pkl", 'rb')  
modelresults = pickle.load(dbfile) 
dbfile.close() 

In [3]:
# print(len(modelresults))
# aiter=iter(modelresults.keys())
# akey=next(aiter)
# print(akey)
# modelresults[akey]

In [4]:
# LOAD LABELS by human
labelsbyhumanpath=Path('/scratch/enis/data/nna/labeling/results/')
# filter by username
labelsbyhuman=[i for i in listdir(labelsbyhumanpath) if (".csv" in i and "Enis" in i) ]

humanresults={}
counter=0
for apath in labelsbyhuman:
    with open(labelsbyhumanpath / apath, newline='') as f:
        reader=csv.reader(f)
        for row in reader:
            counter+=1
            humanresults[row[0]]=row[1:]

print("unique files:",len(humanresults),"\ntotal files",counter)

unique files: 601 
total files 609


In [5]:
histogram={}
for file_name in humanresults:
    place_name=file_name.split("_")[0]
#     print(file_name)
    histogram.setdefault(place_name,0)
    histogram[place_name]+=1

In [6]:
a=list(histogram.items())
a.sort(key=lambda x:x[1],reverse=True)
# a

In [7]:
# load name of the labels
labels=load_labels()

In [8]:
items={}
from IPython.display import Audio,display,HTML
from ipywidgets import Output

items["mp3_output"]=Output()

def play_html_modify(mp3file,items={}):
#     out=items["mp3_output"]
#     displayed=display(HTML("<audio controls  loop autoplay><source src={} type='audio/mpeg'></audio>".format(mp3file)))
    displayed=display(HTML("<audio controls  loop><source src={} type='audio/mpeg'></audio>".format(mp3file)))


In [9]:
def vectorized_y_true(humanresults,tag_set):
    y_true={tag: [None]*len(humanresults) for tag in tag_set}
    for i,tags in enumerate(humanresults.values()):
        # we  only look for tags in tag_set
        for tag in tag_set:
            if tag in tags:
                y_true[tag][i] = 1
            else:
                y_true[tag][i] = 0 
    return y_true

In [10]:
y_true_dict=vectorized_y_true(humanresults,tag_set)


In [11]:
# for i,filename in enumerate(humanresults.keys()):
#     orig,audioop = modelresults[filename]


In [12]:
split_path=Path('/scratch/enis/data/nna/labeling/split_embeddings/')
# filter by username
split_embeds=[i for i in listdir(split_path) ]
raw_embeds =  [i for i in split_embeds if "rawembed" in i]
proc_embeds = [i for i in split_embeds if "_embed" in i]

In [13]:
len(proc_embeds),len(raw_embeds),len(split_embeds)

(10439, 10439, 20878)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.datasets import make_moons, make_circles, make_classification

from sklearn.metrics import roc_auc_score


In [2]:

# h = .02  # step size in the mesh

# names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
#          "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
#          "Naive Bayes", "QDA"]

# classifiers = [
#     KNeighborsClassifier(3),
#     SVC(probability=True,kernel="linear", C=0.025),
#     SVC(probability=True,gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#     MLPClassifier(alpha=1, max_iter=1000),
#     AdaBoostClassifier(),
#     GaussianNB(),
#     QuadraticDiscriminantAnalysis()]



In [3]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
    
short_names=['N. Neighbors',
 'Linear SVM',
 'RBF SVM',
 'Gaussian P.',
 'Decision T.',
 'Random F.',
 'NN',
 'AdaBoost',
 'Naive B.',
 'QDA']

def print_result(results):
    headers= ["Tag",]+short_names
    table=[]
#     sample_count=len(next(iter(y_true_dict.values())))
#     print("Total sample:",sample_count,"And threshold is",prob_threshold)
    for tag in (tag_set):
#         positive_count=sum(y_true_dict[tag])
        table.append([tag,*results[tag]])

    print(tabulate(table, headers=headers))

def reverse_results(results):
    results_clf2={}
    for tag,scores_per_Tag2 in results.items():
        scores_per_Tag=scores_per_Tag2[:]
        maximum=max(scores_per_Tag)
        scores_per_Tag=[color.BOLD + x + color.END if x==maximum else x for x in scores_per_Tag]

        for i,name in enumerate(names):
            score=scores_per_Tag[i]
            results_clf2.setdefault(name,[])
            results_clf2[name].append(score)      
    return results_clf2
        

def print_result2(results):
    results_clf=reverse_results(results)
    headers= ["Tag",]+list(tag_set)
    table=[]
#     sample_count=len(next(iter(y_true_dict.values())))
#     print("Total sample:",sample_count,"And threshold is",prob_threshold)
#     for tag in tag_set:
#         aresult=[]
    for i,name in enumerate(names):
#         positive_count=sum(y_true_dict[tag])
        table.append([short_names[i],*results_clf[name]])
    
    print(tabulate(table, headers=headers))


In [4]:
def get_results(X_input,y_true_dict,classifiers,names,tag_set,logs=False):
    results={}
    for tag in (tag_set):
        X_train, X_test, y_train, y_test = train_test_split(
            X_input, y_true_dict[tag], test_size=0.2, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.25, random_state=42)

        for name, clf in zip(names, classifiers):
            try:
                clf.fit(X_train, y_train)
                y_val_pred=clf.predict_proba(X_val)
                score=roc_auc_score(y_val,y_val_pred[:,1:])
    #             score = clf.score(X_val, y_val)
                score="{:.2}".format(score)

                results.setdefault(tag,[])

                results[tag].append(score)

                if logs : print(tag,name,score) 
            except:
                score="-1"

                results.setdefault(tag,[])

                results[tag].append(score)

                if logs : print(tag,name,score) 
    return results

In [5]:
def map_reduce(X,func_type):
    if func_type=="Average":
        return np.mean(X,axis=1)
    elif func_type=="Concat":
        return np.reshape(X,(601,-1))
    else:
        raise Exception("ERROR with embed type")

def pick_embed(embed_type):
    # humanresults[proc_embeds[0].replace("_embed.npy",".mp3")]
    X=[]
    for i in humanresults:
        if embed_type=="Raw":
            file_name=i.replace(".mp3","_rawembed.npy")
        elif embed_type=="Normalized":
            file_name=i.replace(".mp3","_embed.npy")
        else:
            raise Exception("ERROR with embed type")
        an_x=np.load(split_path / file_name)
        X.append(an_x)
    return X

In [6]:
# params
embed_types=["Raw","Normalized"] # "Raw" or "Normalized"
map_reduce_embeds= ["Average","Concat"]

embed_type=embed_types[0]
map_reduce_type=map_reduce_embeds[0]



In [45]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

all_results={}
for embed_type in embed_types:
    for map_reduce_type in map_reduce_embeds:
        X=pick_embed(embed_type)
        X=map_reduce(X,map_reduce_type)
        print("\n {} embeddings and merged by {} ".format(embed_type,map_reduce_type))
        results=get_results(X,y_true_dict,classifiers,names,tag_set,logs=False)
        all_results[embed_type+"_"+map_reduce_type]=results
        print_result2(results)

Raw embeddings and merged by Average 

Tag             Songbird    Water Bird    Insect    Running Water    Rain    Cable    Wind    Vehicle    Aircraft
------------  ----------  ------------  --------  ---------------  ------  -------  ------  ---------  ----------
N. Neighbors        0.72          0.53      0.53             0.71    0.82     0.8     0.91         -1       0.5
Linear SVM          0.89          0.75      0.74             0.91    0.82     0.84    0.87         -1       0.059
RBF SVM             0.83          0.67      0.48             0.74    0.81     0.84    0.9          -1       0.87
Gaussian P.         0.87          0.65      0.72             0.94    0.8      0.91    0.91         -1       0.77
Decision T.         0.7           0.57      0.58             0.85    0.64     0.8     0.76         -1       0.5
Random F.           0.74          0.62      0.67             0.68    0.86     0.75    0.86         -1       0.34
NN                  0.88          0.72      0.7         

In [47]:
for embed_type in embed_types:
    for map_reduce_type in map_reduce_embeds:
        results=all_results[embed_type+"_"+map_reduce_type]
        print("\n{} embeddings and merged by {} ".format(embed_type,map_reduce_type))
        print_result2(results)


Raw embeddings and merged by Average 
Tag             Songbird    Water Bird    Insect    Running Water    Rain    Cable    Wind    Vehicle    Aircraft
------------  ----------  ------------  --------  ---------------  ------  -------  ------  ---------  ----------
N. Neighbors        0.72          0.53      0.53             0.71    0.82     0.8     0.91         -1       0.5
Linear SVM          0.89          0.75      0.74             0.91    0.82     0.84    0.87         -1       0.059
RBF SVM             0.83          0.67      0.48             0.74    0.81     0.84    0.9          -1       0.87
Gaussian P.         0.87          0.65      0.72             0.94    0.8      0.91    0.91         -1       0.77
Decision T.         0.7           0.57      0.58             0.85    0.64     0.8     0.76         -1       0.5
Random F.           0.74          0.62      0.67             0.68    0.86     0.75    0.86         -1       0.34
NN                  0.88          0.72      0.7         

In [ ]:
# tag="Songbird"
# X_train, X_test, y_train, y_test = train_test_split(
#     X_mean, y_true_dict[tag], test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train, y_train, test_size=0.25, random_state=42)

# for name, clf in zip(names, classifiers):
# #         ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
#     print(name)

#     try:
#         clf.fit(X_train, y_train)
#         y_val_pred=clf.predict_proba(X_val)
#         score=roc_auc_score(y_val,y_val_pred[:,1:])
#         print(y_val_pred[:10,1:])
#         print(clf.classes_)
# #         score = clf.score(X_val, y_val)
#     except:
#         print("ERROR")
        

In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size))

In [ ]:
# import torch
# from data import *
# from model import *
import random
import time
import math

n_hidden = 128
n_epochs = 100000
print_every = 5000
plot_every = 1000
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def categoryFromOutput(output):
    top_n, top_i = output.data.topk(1) # Tensor out of Variable with .data
    category_i = top_i[0][0]
    return all_categories[category_i], category_i

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = Variable(torch.LongTensor([all_categories.index(category)]))
    line_tensor = Variable(lineToTensor(line))
    return category, line, category_tensor, line_tensor

rnn = RNN(n_letters, n_hidden, n_categories)
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    optimizer.zero_grad()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()

    optimizer.step()

    return output, loss.data[0]

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for epoch in range(1, n_epochs + 1):
    category, line, category_tensor, line_tensor = randomTrainingPair()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print epoch number, loss, name and guess
    if epoch % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (epoch, epoch / n_epochs * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if epoch % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

torch.save(rnn, 'char-rnn-classification.pt')

In [ ]:
# from model import *
# from data import *
import sys

rnn = torch.load('char-rnn-classification.pt')

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    
    return output

def predict(line, n_predictions=3):
    output = evaluate(Variable(lineToTensor(line)))

    # Get top N categories
    topv, topi = output.data.topk(n_predictions, 1, True)
    predictions = []

    for i in range(n_predictions):
        value = topv[0][i]
        category_index = topi[0][i]
        print('(%.2f) %s' % (value, all_categories[category_index]))
        predictions.append([value, all_categories[category_index]])

    return predictions

if __name__ == '__main__':
    predict(sys.argv[1])